In [9]:
import pandas as pd
import os

## Extract Audio from Videos

In [ ]:
!apt-get install -y ffmpeg


Reading package lists... Done
Building dependency tree... Done

ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
folder_path = 'your_folder_path_here'
file_names = os.listdir(folder_path)

# Optionally, filter out only files
video_files = [f for f in file_names if os.path.isfile(os.path.join(folder_path, f))]

print(video_files)


In [ ]:
# Create a directory for the audio files
os.makedirs('audio_folder', exist_ok=True)

# Iterate over uploaded files and extract audio
for video in video_files:
    output_audio_file = f'audio_folder/{os.path.splitext(video)[0]}.wav'
    !ffmpeg -i "{video}" -q:a 0 -map a "{output_audio_file}"


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
from google.colab import files
!zip -r output_audio.zip output_audio
# files.download('output_audio.zip')


updating: output_audio/ (stored 0%)
updating: output_audio/beta01_p4_pp_day2.wav (deflated 37%)
updating: output_audio/beta01_p4_pp_day1.wav (deflated 45%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## audio processing

## Use Yingbo's code
### code is adopted from here: https://github.com/yingbo-ma/Predicting-Peer-Satisfaction-EDM2022/blob/main/Features/audio_feature_extraction.py

In [5]:
pip install audiofile


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.2 MB/s eta 0:00:00


In [6]:
pip install opensmile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.2/996.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.0/324.0 kB 22.5 MB/s eta 0:00:00


In [7]:
# Python codes for extracting audio-based features described in Section 4.1
import audiofile
import opensmile

In [ ]:
# notes with YM:
# loudness - every 500 ms take the average
# mfcc - representation of variation + loudness and pitch
# embedding networks for audio - wav2vec

In [22]:
folder_path = 'audio_folder'
audio_files = os.listdir(folder_path)

print(audio_files)

['beta01_p4_pp_day1.wav', 'beta01_p4_pp_day2.wav']


In [24]:
%cd audio_folder

/content/audio_folder


In [25]:
# Directory to save the output CSV files
# Create the output directory if it doesn't exist
output_directory = "audio_features"  # Change this to your desired output directory
os.makedirs(output_directory, exist_ok=True)



# Loop through each audio file
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02, # select feature set
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors
)

Features for beta01_p4_pp_day1.wav saved to beta01_p4_pp_day1_low_level_features.csv
Features for beta01_p4_pp_day2.wav saved to beta01_p4_pp_day2_low_level_features.csv


In [ ]:

for audio_file in audio_files:



  signal, sampling_rate = audiofile.read(audio_file) # read an audio file

   # Process the signal
  features = smile.process_signal(signal, sampling_rate)
  # df is the lowlevel descriptor
  df = pd.DataFrame(features)
  # summary table is the summary of it
  summary_table = features.describe()

  # Create a CSV file name based on the audio file name
  csv_file_name_1 = os.path.splitext(audio_file)[0] + '_low_level_features.csv'
  csv_file_name_2 = os.path.splitext(audio_file)[0] + '_summary_table.csv'

  # Save the DataFrame to a CSV file
  df.to_csv(csv_file_name_1, index=True)
  summary_table.to_csv(csv_file_name_2, index=True)
  print(f"Features for {audio_file} saved to {csv_file_name_1}")
